# Setup

In [1]:
# Imports

import cv2
import keras
import numpy as np
from scipy import sparse
from scipy import ndimage
import pandas as pd
import networkx as nx
from networkx_viewer import Viewer
from sklearn.feature_extraction import image
from matplotlib import colors as mcolors
from multiprocessing import Pool
import random
from itertools import count
import itertools
import matplotlib
import copy
import os.path
import os
import time
import tensorflow as tf
import logging
from sys import stdout
import re
from heapq import heappush, heappop
import scipy.misc
from IPython.display import clear_output
import keras.backend as K

import matplotlib.pyplot as plt
#%matplotlib inline

from keras.models import Sequential
from keras.layers import Conv2D, recurrent, Flatten, Dense, Activation, Recurrent, ZeroPadding2D
from keras import optimizers

Using TensorFlow backend.


# Image to graph

In [2]:
def _make_edges_3d(n_x, n_y, n_z=1):
    """Returns a list of edges for a 3D image.
    Parameters
    ===========
    n_x: integer
        The size of the grid in the x direction.
    n_y: integer
        The size of the grid in the y direction.
    n_z: integer, optional
        The size of the grid in the z direction, defaults to 1
    """
    vertices = np.arange(n_x * n_y * n_z).reshape((n_x, n_y, n_z))
    edges_deep = np.vstack((vertices[:, :, :-1].ravel(),
                            vertices[:, :, 1:].ravel()))
    edges_right = np.vstack((vertices[:, :-1].ravel(),
                             vertices[:, 1:].ravel()))
    edges_down = np.vstack((vertices[:-1].ravel(), vertices[1:].ravel()))
    edges = np.hstack((edges_deep, edges_right, edges_down))
    return edges

In [3]:
def _compute_altitude_3d(edges, img):
    n_x, n_y, n_z = img.shape
    gradient = np.abs(img[edges[0] // (n_y * n_z),
                          (edges[0] % (n_y * n_z)) // n_z,
                          (edges[0] % (n_y * n_z)) % n_z] -
                          img[edges[1] // (n_y * n_z),
                          (edges[1] % (n_y * n_z)) // n_z,
                          (edges[1] % (n_y * n_z)) % n_z])
    return gradient

In [4]:
def img_to_graph(image):
    
    dtype=None
    image = np.atleast_3d(image)
    n_x, n_y, n_z = image.shape
    edges = _make_edges_3d(n_x, n_y, n_z)
    weights = _compute_altitude_3d(edges, image)
    diag = image.ravel()
    n_voxels = diag.size
    diag_idx = np.arange(n_voxels)
    i_idx = np.hstack((edges[0], edges[1]))
    j_idx = np.hstack((edges[1], edges[0]))
    matrix = sparse.coo_matrix((np.hstack((weights, weights, diag)),
                              (np.hstack((i_idx, diag_idx)),
                               np.hstack((j_idx, diag_idx)))),
                              (n_voxels, n_voxels),
                              dtype=dtype)
    graph = nx.from_scipy_sparse_matrix(matrix)
    graph.remove_edges_from(graph.selfloop_edges())
    
    mapping = map_node(image)
    values = get_altitude_map(image)
    
    graph = nx.relabel_nodes(graph, mapping)
    nx.set_node_attributes(graph,'value',values=values)
    
    return graph

In [5]:
def get_altitude_map(img):
    values = dict()
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            values[(row,col)] = img[row,col][0]
    
    return values

In [6]:
def map_node(img):
    
    assert isinstance(img, np.ndarray), "Not an image"
    
    mapping = dict()
    
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            mapping[row * img.shape[1] + col] = ((row), (col))

    return mapping

In [7]:
def get_image_positions(img):
    positions = dict()
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            positions[(row,col)] = (col,row)
    return positions

In [8]:
def view_graph(img, graph, pixel_values=False, 
               figurename='graph.jpg'):
    node_size = 1000
    node_color = 'b'
    seed_color = 'r'
    colors = ["pink","orange","brown","red","green",
              "orange","beige","turquoise","blue"]
    
    plt.figure(figsize=(img.shape[0],img.shape[1]))
    
    positions = get_image_positions(img)
          
    pos = nx.spring_layout(graph,pos=positions, fixed = graph.nodes())
    labels = nx.get_edge_attributes(graph,'weight')
    

    assigned_nodes = list((n for n in graph if graph.node[n]['seed'] != 'none'))
    unassigned_nodes = list((n for n in graph if graph.node[n]['seed'] == 'none'))

    
    seeds = list((n for n in graph if graph.node[n]['type']=='seed'))
    for x in range(len(seeds)):
        #print "Seed {} is {}".format(seeds[x], colors[x])
        print x + 1 % len(colors)
        nodes = list((n for n in graph if graph.node[n]['seed']==seeds[x]))
        nx.draw_networkx_nodes(graph,pos,
                       nodelist=nodes,
                       node_color=colors[(x % (len(colors) - 1)) + 1],
                       node_size=node_size,
                   alpha=0.8)
        nx.draw_networkx_nodes(graph,pos,
                       nodelist=seeds,
                       node_color=colors[(x % (len(colors) - 1)) + 1],
                       node_size=node_size,
                   alpha=0.8, node_shape= 's')
    
    nx.draw_networkx_nodes(graph,pos,
                       nodelist=unassigned_nodes,
                       node_color=colors[0],
                       node_size=node_size,
                   alpha=0.8)

        
    
    if pixel_values:
        values = nx.get_node_attributes(graph,'value')
        nx.draw_networkx_labels(graph,pos,labels=values)
    else:
        nx.draw_networkx_labels(graph,pos)

    nx.draw_networkx_edge_labels(graph,pos,edge_labels=labels)
    nx.draw_networkx_edges(graph,pos , width=5,edge_color='b')
    
    plt.savefig(figurename, dpi=100)
    
    plt.imshow(img, cmap = 'gray')
    
    plt.savefig(figurename, dpi=100)

In [9]:
def plant_seeds(graph, seeds):
    
    temp_graph = graph.copy()
    
    types_dict = dict()
    assignment_dict = dict()
    assignment_history = dict()

    for x in temp_graph.nodes():
        if x in seeds:
            types_dict[x] = "seed"
            assignment_dict[x] = x
            assignment_history[x] = []
        else:
            types_dict[x] = "node"
            assignment_dict[x] = 'none'
            assignment_history[x] = []
            
    nx.set_node_attributes(temp_graph, "type", types_dict)
    nx.set_node_attributes(temp_graph, "seed", assignment_dict)
    nx.set_node_attributes(temp_graph, "assignment history", assignment_history)
    
    return temp_graph

In [10]:
def generate_seeds(graph, n_seeds):
    
    seeds = []
    
    for x in range(n_seeds):
        seeds.append(random.choice(graph.nodes()))
        
    return seeds

In [11]:
def get_graph(window_size, window_tl, image):
    
    fname = "Saved_Graphs/graph_{}_{}_{}_{}.p".format(window_tl[0],window_tl[1],window_size[0],window_size[1])
    
    #if graph already exists, load graph
    if os.path.isfile(fname):
        
        print "Retrieveing graph."
        graph = nx.read_gpickle(fname)
        
    #else, build the graph    
    else:
        
        print "Graph doesn't exist. Creating graph."
        #Translate image to 4 connected grid graph
        graph = img_to_graph(image)
        nx.write_gpickle(graph,fname)
        
    return graph

In [12]:
def get_graph_seed_assignments(graph):
    assignments = dict()
    c = count(0)
    for seed in seeds:
        a = next(c)
        nodes = list((n for n in graph if graph.node[n]['seed']==seeds[a]))
        assignments.update(dict.fromkeys(nodes, a))
    return assignments

In [13]:
def get_spaced_colors(n):
    max_value = 16581375 #255**3
    interval = int(max_value / n)
    colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]
    
    return np.array([(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in colors])

In [14]:
def projections(graph, seed, image):
    '''This will output the relative labelings and save the RGB image to the relative labelings folder'''
    
    positions = get_image_positions(image)
    assignments = get_graph_seed_assignments(graph)
    
    seeds = set(assignments.values())
    n_seeds = len(seeds)
    
    seed_colors = get_spaced_colors(n_seeds)
    
    mask = np.zeros((image.shape[0], image.shape[1], 3))
    
    s = pd.Series(assignments)
    s = s.values.reshape(image.shape)
    
    for x in range(n_seeds):
        a = s == x
        if x == seed:
            mask[a] = [0, 255, 0]
        else:
            mask[a] = [0, 0, 255]
            
    print mask.shape
    return mask

In [15]:
def prims_msf(graph):
    iters = []
    G = graph.copy()
    MSF = nx.Graph()
    nodes = G.nodes()
    s = filter(lambda (n, d): d['type'] == 'seed', G.nodes(data=True))
    seeds = []
    iteration = count(0)
    for x in s:
        seeds.append(x[0])
    push = heappush
    pop = heappop

    while nodes:
        frontier = []
        visited = []
        for u in seeds:
            nodes.remove(u)
            visited.append(u)
            
            # Add seed to MSF
            MSF.add_node(u, attr_dict=G.node[u])
            
            # Push all edges
            for u, v in G.edges(u):
                push(frontier, (G[u][v].get('weight', 1), u, v))
        
        

        while frontier:
            i = next(iteration)
            W, u, v = pop(frontier)   

            if v in visited:
                continue
            
            # Assign the node
            G.node[v]['seed'] = G.node[u]['seed']
            
            # Add node and edge to MSF
            MSF.add_node(v,attr_dict=G.node[v])
            MSF.add_edge(u,v,attr_dict=G.edge[u][v])
            
            G.node[G.node[u]['seed']]['assignment history'] = G.node[G.node[u]['seed']]['assignment history'] + [v]       
            
            visited.append(v)
            nodes.remove(v)
            for v, w in G.edges(v):
                if not w in visited:
                    push(frontier, (G[v][w].get('weight', 1), v, w))
                    
            if i % 10 == 0:
                #view_graph(img, sub_G, figurename="gif/iteration_{}".format(i))
                continue
    return MSF

In [16]:
def field_of_view(master_image, center_pixel, crop_size):
    """Returns a crop of the master image centered on a specified pixel.
    
    Args:
        master_image (numpy.ndarray): The master image to be cropped.
        center_pixel (tuple): The pixel to be at the center of the crop.
        window_size (tuple): The dimensions of the newly cropped image.
        
    Returns:
        A cropped image of the master image with the specified pixel at the center.
        
    Examples:
        fov = field_of_view(I, (0,0), (50,50))"""
    
    center = (center_pixel[0] + crop_size[0],center_pixel[1] + crop_size[1])

    t_l = (center[0] - crop_size[0]//2, center[1] - crop_size[1]//2)
    
    return master_image[t_l[0]:t_l[0] + crop_size[0],t_l[1]:t_l[1] + crop_size[1],:]

In [17]:
def assignment_mask(image, graph):
    positions = get_image_positions(image)
    assignments = get_graph_seed_assignments(graph)
    
    seeds = set(assignments.values())
    n_seeds = len(seeds)
    
    seed_colors = get_spaced_colors(n_seeds)
    
    mask = np.zeros((image.shape[0], image.shape[1], 3))
    
    s = pd.Series(assignments)
    s = s.values.reshape(image.shape)
    
    for x in range(n_seeds):
        a = s == x
        mask[a] = seed_colors[x]
    print seed_colors[0]
    return mask

In [18]:
def max_arc_topographical_distance(path, graph):
    """Returns the max-arc topographical distance of a path in a graph.
    
    Args:
        path (list): A list of tuples defining the path traversed in graph.
        graph (networkx.classes.graph.Graph): The graph containing the path.
        
    Returns:
        Returns the max-arc topographical distance of path."""
    
    G = graph.subgraph(path)
    
    high = -np.infty
    
    for x, y in G.edges_iter():
        weight = G.edge[x][y]['weight']
        if weight > high:
            high = weight
            
    return high

In [19]:
def get_boundary_probabilities(img):
    """Returns boundary probabilites for an image"""
    
    #Make a placeholder for boundary probabilities coming in next iteration of pipeline
    return np.random.rand(img.shape[0],img.shape[1])

# Start

In [20]:
# Set image size
window_tl = (50,30)
window_size = (32, 32)
fov_size = (32, 32)

#import Images
img = cv2.imread('1O.jpg', 0)
gt = cv2.imread('1G.jpg', 0)

#resize
img = img[window_tl[0]:window_tl[0] + window_size[0],
          window_tl[1]:window_tl[1] + window_size[1]]
gt = gt[window_tl[0]:window_tl[0] + window_size[0],
        window_tl[1]:window_tl[1] + window_size[1]]

#set type
img = img.astype(np.int16)
gt = gt.astype(np.int16)

#Pad image for field of view
I = np.pad(img,(fov_size[0],fov_size[1]),'reflect')

boundary_probabilities = get_boundary_probabilities(I)

#Augment the image with boundary probabilities
I_a = np.dstack((I, boundary_probabilities))

In [21]:
# Get input images
input_images = dict()

for x in range(window_size[0]):
    for y in range(window_size[1]):
        node = (x, y)
        input_images[node] = field_of_view(I_a, node, fov_size)

In [22]:
#Translate image to 4 connected grid graph
img_graph = img_to_graph(img)
img_graph.name = "image_graph"
ground_truth_graph = img_to_graph(gt)
ground_truth_graph.name = "ground_truth_graph"

In [23]:
#import seeds
seeds = []
f = open("seeds1G.txt", 'r')
for line in f:
    x = int(float(re.split(' ', line)[0]))
    y = int(float(re.split(' ', line)[1]))
    seed = (x, y)
    
    if x >= window_tl[0] and x <= window_tl[0] + window_size[0]:
        if y >= window_tl[1] and y <= window_tl[1] + window_size[1]:
            x = x - window_tl[0]
            y = y - window_tl[1]
            seed = (x, y)
            seeds.append(seed)

In [24]:
#plant seeds
img_graph = plant_seeds(img_graph, seeds)
ground_truth_graph = plant_seeds(ground_truth_graph, seeds)

# Make Model

In [25]:
# This placeholder will contain the input images
image_placeholder = tf.placeholder(tf.float32, shape=(None, window_size[0], window_size[0], 2))

# Create model
m = Conv2D(16, 5, padding = 'same',
                 activation='elu', dilation_rate=1) (image_placeholder)
m = Conv2D(32, 3, padding = 'same',
                 activation='elu', dilation_rate=1) (m)
m = Conv2D(32, 3, padding = 'same',
                 activation='elu', dilation_rate=2) (m)
m = Conv2D(64, 3, padding = 'same',
                 activation='elu', dilation_rate=4) (m)
m = Conv2D(64, 3, padding = 'same',
                 activation='elu', dilation_rate=8) (m)
m = Conv2D(128, 3, padding = 'same',
                 activation='elu', dilation_rate=16) (m)
m = Flatten()(m)
m = Dense(64, activation='relu')(m)
preds = Dense(1)(m)

# Create session
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [50]:
def prims_on_demand(G):
    """
    Creates a minimum spanning forest from give graph.
    
    Args:
        G (Graph): Graph to use for minimum spanning forest.
        
    Returns:
        MSF (Graph): Minimum spanning forest.
    """
    
    MSF = nx.Graph()
    nodes = G.nodes()
    
    s = filter(lambda (n, d): d['type'] == 'seed', G.nodes(data=True))
    seeds = []
    
    iteration = count(1)
    
    for x in s:
        seeds.append(x[0])
        
    push = heappush
    pop = heappop
    
    num_edges = G.number_of_edges()

    while nodes:
        
        frontier = []
        visited = []
        
        for u in seeds:
            nodes.remove(u)
            visited.append(u)
            
            # Add seed to MSF
            MSF.add_node(u, attr_dict=G.node[u])
            
            # Push all edges
            for u, v in G.edges(u):
                
                stdout.write("\rCalculating edge {}/{}".format(next(iteration), num_edges))
                stdout.flush()
                
                # Fetch cropped image from dictionary
                fov = input_images[(v[0], v[1])]
                
                # Calculate newly discovered edge weight
                x = np.expand_dims(fov, axis=0)
                with sess.as_default():
                    weight = sess.run([preds], feed_dict={image_placeholder: x})[0][0][0]
                    
                G.edge[u][v]['weight'] = weight

                push(frontier, (weight, u, v))
        
        

        while frontier:
            W, u, v = pop(frontier)   
            
            if v in visited:
                continue
            
            # assign the node
            G.node[v]['seed'] = G.node[u]['seed']
            
            # add node and edge to MSF
            MSF.add_node(v,attr_dict=G.node[v])
            MSF.add_edge(u,v,attr_dict=G.edge[u][v])

            #p = projections(G, G.node[u]['seed'], image)
            #plt.imshow(p,cmap='jet')
            #savefig('projections_i_{}.png'.format(i))
            G.node[G.node[u]['seed']]['assignment history'] = \
            G.node[G.node[u]['seed']]['assignment history'] + [v]       
            
            visited.append(v)
            nodes.remove(v)
            for v, w in G.edges(v):
                if not w in visited:
                    stdout.write("\rCalculating edge {}/{}".format(next(iteration), num_edges))
                    stdout.flush()
                    
                    # Fetch cropped image from dictionary
                    fov = input_images[(w[0], w[1])]
                    
                    # Calculate newly discovered edge weight
                    x = np.expand_dims(fov, axis=0)
                    with sess.as_default():
                        weight = sess.run([preds], feed_dict={image_placeholder: x})[0][0][0]
                    G.edge[v][w]['weight'] = weight
                
                    push(frontier, (weight, v, w))
                    
            #if i % 10 == 0:
                #view_graph(img, G, figurename="gif/iteration_{}".format(i))
                #continue
    stdout.write("\n")
    
    return MSF

In [52]:
#Get MSF
print "Calculating Image MSF"
img_msf = prims_on_demand(img_graph)
img_msf.name = "Image_MSF"
print "Gt MSF"
gt_msf = prims_msf(ground_truth_graph)
gt_msf.name = "GT_MSF"

#Create lists of cut edges
hummus_star = []
trash = [hummus_star.append(e) if gt_msf.node[e[0]]['seed'] is not gt_msf.node[e[1]]['seed'] else '' for e in ground_truth_graph.edges_iter()]

hummus = []
trash = [hummus.append(e) if img_msf.node[e[0]]['seed'] is not img_msf.node[e[1]]['seed'] else '' for e in img_graph.edges_iter()]

#Remove cut edges from graph
gt_cuts = gt_msf.copy()
gt_cuts.remove_edges_from(hummus)

img_cuts = img_graph.copy()
img_cuts.remove_edges_from(hummus_star)

print "Calculating Constrained MSF"
constrained_msf = prims_on_demand(img_cuts)
constrained_msf.name = "constrained_MSF"

Calculating Image MSF
Calculating edge 1984/1984
Gt MSF
Calculating Constrained MSF
Calculating edge 1933/1933


In [28]:
view_graph(img, img_msf, figurename='img_msf.jpg')

1
2


In [29]:
def max_arc_topographical(graph, path):
    
    G = graph.subgraph(path)
    
    high = -np.infty
    edge = None
    
    for x, y in G.edges_iter():
        weight = G.edge[x][y]['weight']
        if weight > high:
            high = weight
            edge = (x,y)
            
    return high, edge

In [38]:
T_star = []
T = field_of_view(I_a, (0,0), fov_size)
T = np.expand_dims(T, axis=0)

msf = img_msf
cons_msf = constrained_msf

loss = 0
i = 0
a = []

nodes = list((n for n in msf if msf.node[n]['type']=='node'))
for node in nodes:

    m_hat = msf.node[node]['seed']
    m_star = cons_msf.node[node]['seed']

    # Get the path from seed to node
    phi_hat = next(nx.all_simple_paths(msf, m_hat, node))
    phi_star = next(nx.all_simple_paths(cons_msf, m_star, node))

    # Get path distance
    t, t_edge = max_arc_topographical(msf, phi_hat)
    t_star, t_star_edge = max_arc_topographical(cons_msf, phi_star)    
    
    if phi_hat == phi_star:
            continue
    else: 
        
        # Predict the node
        print t_edge
        node = t_edge[0]
        fov = input_images[node]
        x = np.expand_dims(fov, axis=0)
        with sess.as_default():
            weight = sess.run([preds], feed_dict={image_placeholder: x})[0][0][0]
        
        if t != weight:
            node = t_edge[1]
            fov = input_images[node]
            x = np.expand_dims(fov, axis=0)
            with sess.as_default():
                weight = sess.run([preds], feed_dict={image_placeholder: x})[0][0][0]
            if t != weight:
                print "OH SHIT IT'S AN ALIEN!!"
                
        print "{} - {} = {}".format(t_star, t, (t_star - t))
        print "{} - {} = {}".format(t_star, weight, (t_star - weight))
        
        T_star.append(t_star)
        T1 = field_of_view(I_a, node, fov_size)
        T1 = np.expand_dims(T1, axis=0)
        
        T = np.concatenate((T, T1),axis=0)
        
        i = i + 1
        loss = loss + (t_star - t)
        #f_p = img_graph.edge[t_star_edge[0]][t_star_edge[1]]['weight']
        #print t_star, f_p
        #if t_star != f_p:
        #    print "problem"
        #else:
        #    print "no problem"
            
    
    
T = np.delete(T, 0, 0)
print loss

((16, 6), (15, 6))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((3, 25), (4, 25))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((5, 9), (6, 9))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((14, 4), (15, 4))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((3, 25), (4, 25))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((19, 19), (18, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((4, 7), (3, 7))
-0.00551000097767 - 

-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((7, 3), (6, 3))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((13, 22), (12, 22))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((2, 1), (3, 1))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((25, 5), (26, 5))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 20), (0, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((15, 13), (16, 13))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((18, 0), (19, 0))
-0.00551000097767 - -0.00551000097767

-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((15, 27), (14, 27))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 20), (0, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((18, 0), (19, 0))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 23), (0, 22))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((25, 5), (26, 5))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((13, 22), (12, 22))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((3, 25), (4, 25))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.0055100009

((0, 23), (0, 22))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((18, 0), (19, 0))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 20), (0, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((13, 22), (12, 22))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((24, 30), (23, 30))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 20), (0, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((14, 17), (13, 17))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((13, 1), (14, 1))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 12), (0, 11))
-0.005510000

-0.00551000097767 - -0.00551000097767 = 0.0
((8, 1), (9, 1))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 20), (0, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 12), (0, 11))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 20), (0, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((11, 2), (12, 2))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((13, 22), (12, 22))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((4, 7), (3, 7))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((18, 0), (19, 0))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((25, 5), (26, 5))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 =

((22, 23), (21, 23))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 12), (0, 11))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((3, 25), (4, 25))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((11, 2), (12, 2))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((7, 3), (6, 3))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((26, 24), (27, 24))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((1, 21), (2, 21))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((2, 1), (3, 1))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 23), (0, 22))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((13, 22), (12, 22))
-0.0055100009776

-0.00551000097767 - -0.00551000097767 = 0.0
((2, 1), (3, 1))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 1), (0, 0))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 20), (0, 19))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((8, 23), (7, 23))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((1, 21), (2, 21))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((2, 18), (1, 18))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((15, 27), (14, 27))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((0, 27), (0, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((12, 1), (13, 1))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 =

((1, 21), (2, 21))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((11, 16), (10, 16))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((3, 0), (2, 0))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((11, 16), (10, 16))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((10, 11), (9, 11))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((4, 7), (3, 7))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((15, 27), (14, 27))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((2, 1), (3, 1))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((9, 26), (8, 26))
-0.00551000097767 - -0.00551000097767 = 0.0
-0.00551000097767 - -0.00551000097767 = 0.0
((13, 1), (14, 1))
-0.00551000097767

In [39]:
def create_batches(data, batch_size):
    """
    Split data into batches of a specific size.  If there is a remainder of data,
    the last batch might be smaller.
    
    Args:
        data (numpy.ndarray):  Data to be split into batches.
        batch_size (int): Size of batches.
    
    Returns:
        batches (list): A list of batches (or numpy arrays), each the specified batch size. 
        
    Examples:
        >>> batches = create_batches(x_train, 32)
    """
    
    batches = list()
    
    while data.shape[0] >= batch_size:
        batches.append(data[0:batch_size,:,:,:])
        data = np.delete(data, range(0,batch_size), 0)

    batches.append(data[0:data.shape[0],:,:,:])
    data = np.delete(data, range(0,data.shape[0]), 0)
    
    return batches

In [40]:
batches = create_batches(T, 32)

In [41]:
len(batches)

22

In [42]:
# Define cost
cost = tf.reduce_sum(preds)

# Define optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate=0.00001).minimize(cost)

with sess.as_default():
    sess.run(tf.global_variables_initializer())
    for x in batches:
        opt.run(feed_dict={image_placeholder: x})
        best = sess.run([preds], feed_dict={image_placeholder: x})

In [43]:
best

[array([[-0.00676],
        [-0.00676],
        [-0.00676],
        [-0.00676]], dtype=float32)]

In [44]:
#Get MSF
print "Image MSF"
img_msf = prims_on_demand(img_graph)
img_msf.name = "Image_MSF"
print "Gt MSF"
gt_msf = prims_msf(ground_truth_graph)
gt_msf.name = "GT_MSF"

#Create lists of cut edges
hummus_star = []
trash = [hummus_star.append(e) if gt_msf.node[e[0]]['seed'] is not gt_msf.node[e[1]]['seed'] else '' for e in ground_truth_graph.edges_iter()]

hummus = []
trash = [hummus.append(e) if img_msf.node[e[0]]['seed'] is not img_msf.node[e[1]]['seed'] else '' for e in img_graph.edges_iter()]

#Remove cut edges from graph
gt_cuts = gt_msf.copy()
gt_cuts.remove_edges_from(hummus)

img_cuts = img_graph.copy()
img_cuts.remove_edges_from(hummus_star)

print "Constrained MSF"
constrained_msf = prims_on_demand(img_cuts)
constrained_msf.name = "constrained_MSF"

Image MSF
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676


-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-

-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676

Gt MSF
Constrained MSF
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.0

-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-

-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-0.00676
-

In [45]:
view_graph(img, img_msf, figurename='img_msf.jpg')

1
2
